<a href="https://colab.research.google.com/github/OxyzGiaHuy/BCCD/blob/main/BCCD_with_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Shenggan/BCCD_Dataset.git

Cloning into 'BCCD_Dataset'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800 (from 1)
Receiving objects: 100% (800/800), 7.39 MiB | 11.19 MiB/s, done.
Resolving deltas: 100% (378/378), done.


In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00


In [3]:
# -*- coding: utf-8 -*-
import os
import xml.etree.ElementTree as ET
import shutil
from ultralytics import YOLO
from pathlib import Path

# --- Configuration ---
dataset_path = Path('/content/BCCD_Dataset/BCCD')
yolo_data_dir = Path('/content/BCCD_YOLO')

# Class names in your dataset
class_names = ['RBC', 'WBC', 'Platelets']

# YOLOv8 model to use (n=nano, s=small, m=medium, l=large, x=xlarge)
model_size = 'n' # Using nano for a faster demo

# Training parameters
epochs = 50 # Reduce for quicker test, increase for better results
imgsz = 640 # Image size for training/inference
batch_size = 16 # Adjust based on Colab GPU memory

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# --- Setup ---
print("Setting up environment...")

# Install ultralytics
!pip install ultralytics -q

# Create output directory for YOLO data
yolo_data_dir.mkdir(parents=True, exist_ok=True)
(yolo_data_dir / 'images').mkdir(exist_ok=True)
(yolo_data_dir / 'labels').mkdir(exist_ok=True)

Setting up environment...


In [5]:
# --- Dataset Conversion (VOC XML to YOLO TXT) ---

def convert_voc_to_yolo(xml_dir, img_dir, label_dir, class_names):
    """Converts VOC XML annotations to YOLO TXT format."""
    print(f"Converting annotations from {xml_dir} to YOLO format in {label_dir}")
    class_map = {name: i for i, name in enumerate(class_names)}
    skipped_files = []

    for xml_file in os.listdir(xml_dir):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(xml_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        img_filename = root.find('filename').text
        # BCCD uses .jpg extension
        img_path = os.path.join(img_dir, img_filename)
        if not os.path.exists(img_path):
            # Try alternative extension if original doesn't exist (e.g. if it was .png)
             img_path = os.path.join(img_dir, os.path.splitext(img_filename)[0] + '.jpg')
             if not os.path.exists(img_path):
                 print(f"Warning: Image file not found for {xml_file} at expected paths. Skipping.")
                 skipped_files.append(xml_file)
                 continue

        try:
            size = root.find('size')
            img_width = int(size.find('width').text)
            img_height = int(size.find('height').text)
        except:
            print(f"Warning: Could not parse image size for {xml_file}. Skipping.")
            skipped_files.append(xml_file)
            continue


        yolo_txt_filename = os.path.splitext(xml_file)[0] + '.txt'
        yolo_txt_path = os.path.join(label_dir, yolo_txt_filename)

        with open(yolo_txt_path, 'w') as f:
            for obj in root.findall('object'):
                obj_name = obj.find('name').text
                if obj_name not in class_names:
                    print(f"Warning: Unknown class '{obj_name}' found in {xml_file}. Skipping object.")
                    continue

                class_id = class_map[obj_name]
                bndbox = obj.find('bndbox')
                xmin = int(float(bndbox.find('xmin').text))
                ymin = int(float(bndbox.find('ymin').text))
                xmax = int(float(bndbox.find('xmax').text))
                ymax = int(float(bndbox.find('ymax').text))

                # Convert to YOLO format (x_center, y_center, width, height) normalized
                x_center = (xmin + xmax) / 2.0 / img_width
                y_center = (ymin + ymax) / 2.0 / img_height
                width = (xmax - xmin) / float(img_width)
                height = (ymax - ymin) / float(img_height)

                # Ensure values are within [0, 1] and positive
                x_center = max(0.0, min(1.0, x_center))
                y_center = max(0.0, min(1.0, y_center))
                width = max(0.0, width)
                height = max(0.0, height)
                width = min(1.0, width) # Handle cases where box might exceed image slightly due to rounding
                height = min(1.0, height)


                if width > 0 and height > 0: # Only write if bounding box is valid
                     f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                else:
                    print(f"Warning: Invalid bounding box dimensions for object '{obj_name}' in {xml_file}. Skipping object.")


    print(f"Conversion complete. Skipped {len(skipped_files)} files due to errors.")


# Check if source dataset exists
if not dataset_path.exists():
    print(f"Error: Dataset not found at {dataset_path}")
    print("Please upload or symlink the BCCD folder to /content/BCCD")
    # Example if using Google Drive:
    # from google.colab import drive
    # drive.mount('/content/drive')
    # dataset_path = Path('/content/drive/My Drive/path/to/your/BCCD') # Adjust path
    # If you copy: !cp -r "/content/drive/My Drive/path/to/your/BCCD" "/content/"
    exit() # Stop execution if dataset not found


# Perform conversion
convert_voc_to_yolo(
    xml_dir=dataset_path / 'Annotations',
    img_dir=dataset_path / 'JPEGImages',
    label_dir=yolo_data_dir / 'labels',
    class_names=class_names
)

Converting annotations from /content/BCCD_Dataset/BCCD/Annotations to YOLO format in /content/BCCD_YOLO/labels
Conversion complete. Skipped 0 files due to errors.


In [9]:
# --- Prepare Train/Val/Test Splits ---

# Create train/val/test directories inside images and labels
for split in ['train', 'val', 'test']:
    (yolo_data_dir / 'images' / split).mkdir(parents=True, exist_ok=True)
    (yolo_data_dir / 'labels' / split).mkdir(parents=True, exist_ok=True)

# Read splits from ImageSets
print("Copying images and labels according to splits...")
splits = {}
for split_file in ['train', 'val', 'test']:
    split_path = dataset_path / 'ImageSets' / 'Main' / f'{split_file}.txt'
    if not split_path.exists():
         print(f"Warning: Split file not found: {split_path}. Skipping {split_file} split.")
         splits[split_file] = []
         continue

    with open(split_path, 'r') as f:
        splits[split_file] = [line.strip().split()[0] for line in f.readlines() if line.strip()]

    for img_id in splits[split_file]:
        # Assuming image files are .jpg and corresponding labels are .txt
        img_src = dataset_path / 'JPEGImages' / f'{img_id}.jpg'
        label_src = yolo_data_dir / 'labels' / f'{img_id}.txt' # Look for converted label
        img_dest = yolo_data_dir / 'images' / split_file / f'{img_id}.jpg'
        label_dest = yolo_data_dir / 'labels' / split_file / f'{img_id}.txt'

        # Check if image and label exist before copying
        if img_src.exists():
            shutil.copy(img_src, img_dest)
        else:
            print(f"Warning: Image file not found for {img_id} in {split} split. Skipping.")

        if label_src.exists():
            shutil.copy(label_src, label_dest)
        else:
             # This can happen if the original XML had issues or no objects
            print(f"Warning: Label file not found for {img_id} in {split} split. Skipping label copy.")


print("Dataset preparation complete.")

Copying images and labels according to splits...
Dataset preparation complete.


In [7]:
# --- Create data.yaml file ---
data_yaml_content = f"""
path: {yolo_data_dir}
train: images/train
val: images/val
test: images/test

nc: {len(class_names)}
names: {class_names}
"""

data_yaml_path = yolo_data_dir / 'data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"Created data.yaml at {data_yaml_path}:\n{data_yaml_content}")

# --- Load and Train YOLOv8 Model ---
print(f"Loading YOLOv8{model_size} model...")
model = YOLO(f'yolov8{model_size}.pt')  # Load a pre-trained model

print(f"Starting training for {epochs} epochs...")
results = model.train(
    data=str(data_yaml_path),
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    name=f'yolov8{model_size}_bccd' # Name for the run
)

print("Training finished.")

Created data.yaml at /content/BCCD_YOLO/data.yaml:

path: /content/BCCD_YOLO
train: images/train
val: images/val
test: images/test

nc: 3
names: ['RBC', 'WBC', 'Platelets']

Loading YOLOv8n model...


100%|██████████| 6.25M/6.25M [00:00<00:00, 95.5MB/s]


Starting training for 50 epochs...
Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/BCCD_YOLO/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_bccd, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

100%|██████████| 755k/755k [00:00<00:00, 20.9MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      50/50      4.07G     0.8848     0.6427      1.052        171        640: 100%|██████████| 13/13 [00:02<00:00,  4.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.70it/s]

                   all         87       1137      0.869      0.884      0.914       0.64



50 epochs completed in 0.061 hours.
Optimizer stripped from runs/detect/yolov8n_bccd/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8n_bccd/weights/best.pt, 6.2MB

Validating runs/detect/yolov8n_bccd/weights/best.pt...
Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


                   all         87       1137      0.879      0.878      0.916      0.644
                   RBC         83        967      0.813      0.799      0.883      0.634
                   WBC         86         87      0.983          1      0.988      0.825
             Platelets         45         83      0.842      0.834      0.878      0.473
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/yolov8n_bccd
Training finished.


In [8]:
# --- Evaluate Model ---
print("Evaluating model on the test set...")
# The val set is evaluated by default after training, but let's explicitly run test
test_results = model.val(data=str(data_yaml_path), split='test')

print("\n--- Evaluation Results (Test Set) ---")
print(f"mAP50: {test_results.box.map50}")
print(f"mAP50-95: {test_results.box.map}")
print(f"Results saved to {model.trainer.save_dir}")


# --- Optional: Run Inference on a Test Image ---
print("\n--- Running Inference Example ---")
if splits['test']:
    example_img_id = splits['test'][0]
    example_img_path = yolo_data_dir / 'images' / 'test' / f'{example_img_id}.jpg'
    if example_img_path.exists():
        print(f"Running inference on {example_img_path}")
        predict_results = model(str(example_img_path))
        # The predict results contain boxes, scores, classes etc.
        # You can print or visualize these results
        print("Inference results (example):")
        for r in predict_results:
            print(f"  Boxes: {len(r.boxes)} detected")
            # r.boxes contains Boxes object with xyxy, conf, cls
            # r.save_txt() # Save results to a txt file
            # r.save_dir # Directory where image with predictions might be saved

        # To visualize, you might need to save the image with detections
        # model.predict(str(example_img_path), save=True, exist_ok=True)
        # print(f"Example prediction image potentially saved to runs/detect/predict*/{example_img_id}.jpg")

    else:
         print(f"Example test image not found at {example_img_path}. Cannot run inference example.")
else:
    print("No images found in test split. Cannot run inference example.")

print("\nSimple YOLOv8 script finished.")

Evaluating model on the test set...
Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 829.2±216.5 MB/s, size: 19.9 KB)


val: Scanning /content/BCCD_YOLO/labels/test... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<00:00, 1931.86it/s]

val: New cache created: /content/BCCD_YOLO/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.26it/s]


                   all         72        945      0.826      0.894        0.9      0.621
                   RBC         69        805      0.758      0.811      0.868      0.626
                   WBC         68         71      0.936          1      0.963       0.78
             Platelets         39         69      0.785       0.87       0.87      0.458
Speed: 6.0ms preprocess, 7.6ms inference, 0.1ms loss, 8.8ms postprocess per image
Results saved to runs/detect/yolov8n_bccd2

--- Evaluation Results (Test Set) ---
mAP50: 0.90026135223957
mAP50-95: 0.621327011797657
Results saved to runs/detect/yolov8n_bccd

--- Running Inference Example ---
Running inference on /content/BCCD_YOLO/images/test/BloodImage_00007.jpg

image 1/1 /content/BCCD_YOLO/images/test/BloodImage_00007.jpg: 480x640 10 RBCs, 1 WBC, 1 Platelets, 54.9ms
Speed: 2.1ms preprocess, 54.9ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
Inference results (example):
  Boxes: 12 detected

Simple YOLOv8 script f